# Data Cleaning

First, we will open the AviationData csv and look at the columns.

In [91]:
import pandas as pd
 
df = pd.read_csv('./data/AviationData.csv', encoding='latin1')

df.head(5)

/var/folders/4s/pf1zdv_x4_qcflpq4m943q0m0000gn/T/ipykernel_75844/3117126247.py:3: DtypeWarning: Columns (6,7,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/AviationData.csv', encoding='latin1')


,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


Then, we will look at the column names and the dataframe info to find possible null values.

In [4]:
df.columns

Index(['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date',
       'Location', 'Country', 'Latitude', 'Longitude', 'Airport.Code',
       'Airport.Name', 'Injury.Severity', 'Aircraft.damage',
       'Aircraft.Category', 'Registration.Number', 'Make', 'Model',
       'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'FAR.Description',
       'Schedule', 'Purpose.of.flight', 'Air.carrier', 'Total.Fatal.Injuries',
       'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured',
       'Weather.Condition', 'Broad.phase.of.flight', 'Report.Status',
       'Publication.Date'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50132 non-null  object 
 9   Airport.Name            52704 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87507 non-null  object 
 14  Make                    88826 non-null

We decided to drop the Latitude, Longitude, Airport Code, and Airport Name columns, instead choosing to use the Location column.

In [6]:
dropped_cols = ['Latitude', 'Longitude', 'Airport.Code', 'Airport.Name', 'Schedule']
dropped_df = df.drop(columns=dropped_cols)
dropped_df = dropped_df.dropna(axis='index', subset=['Location'])
dropped_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88837 entries, 0 to 88888
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88837 non-null  object 
 1   Investigation.Type      88837 non-null  object 
 2   Accident.Number         88837 non-null  object 
 3   Event.Date              88837 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88612 non-null  object 
 6   Injury.Severity         87837 non-null  object 
 7   Aircraft.damage         85650 non-null  object 
 8   Aircraft.Category       32278 non-null  object 
 9   Registration.Number     87477 non-null  object 
 10  Make                    88774 non-null  object 
 11  Model                   88745 non-null  object 
 12  Amateur.Built           88739 non-null  object 
 13  Number.of.Engines       82782 non-null  float64
 14  Engine.Type             81747 non-null  obj

Separated Location column into two new columns: City and State

In [7]:
dropped_df[['Location_City', 'Location_State']] = dropped_df['Location'].str.split(', ', n=1, expand=True)
dropped_df['Location_City'] = dropped_df['Location_City'].str.title()
dropped_df[['Location_City', 'Location_State']].head(10)

,Location_City,Location_State
0,Moose Creek,ID
1,Bridgeport,CA
2,Saltville,VA
3,Eureka,CA
4,Canton,OH
5,Boston,MA
6,Cotton,MN
7,Pullman,WA
8,East Hanover,NJ
9,Jacksonville,FL


Standardized the string format of the Make column.

In [8]:
dropped_df['Make'] = dropped_df['Make'].str.title()
dropped_df['Make'].value_counts()

Make
Cessna           27140
Piper            14865
Beech             5367
Boeing            2734
Bell              2717
                 ...  
Cohen                1
Kitchens             1
Lutes                1
Izatt                1
Royse Ralph L        1
Name: count, Length: 7587, dtype: int64

Let's take another look at dropped_df.

In [9]:
dropped_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88837 entries, 0 to 88888
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88837 non-null  object 
 1   Investigation.Type      88837 non-null  object 
 2   Accident.Number         88837 non-null  object 
 3   Event.Date              88837 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88612 non-null  object 
 6   Injury.Severity         87837 non-null  object 
 7   Aircraft.damage         85650 non-null  object 
 8   Aircraft.Category       32278 non-null  object 
 9   Registration.Number     87477 non-null  object 
 10  Make                    88774 non-null  object 
 11  Model                   88745 non-null  object 
 12  Amateur.Built           88739 non-null  object 
 13  Number.of.Engines       82782 non-null  float64
 14  Engine.Type             81747 non-null  obj

Let's assume that if there are values in the Total Injuries columns that are NaN, they should be zero.

In [10]:
dropped_df[['Total.Fatal.Injuries', 'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured']] = dropped_df[['Total.Fatal.Injuries', 'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured']].fillna(value=0)
dropped_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88837 entries, 0 to 88888
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88837 non-null  object 
 1   Investigation.Type      88837 non-null  object 
 2   Accident.Number         88837 non-null  object 
 3   Event.Date              88837 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88612 non-null  object 
 6   Injury.Severity         87837 non-null  object 
 7   Aircraft.damage         85650 non-null  object 
 8   Aircraft.Category       32278 non-null  object 
 9   Registration.Number     87477 non-null  object 
 10  Make                    88774 non-null  object 
 11  Model                   88745 non-null  object 
 12  Amateur.Built           88739 non-null  object 
 13  Number.of.Engines       82782 non-null  float64
 14  Engine.Type             81747 non-null  obj

Drop Air Carrier column, I feel like Make and Model is a more important statistic.

In [11]:
dropped_df = dropped_df.drop(columns=['Air.carrier'])
dropped_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88837 entries, 0 to 88888
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88837 non-null  object 
 1   Investigation.Type      88837 non-null  object 
 2   Accident.Number         88837 non-null  object 
 3   Event.Date              88837 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88612 non-null  object 
 6   Injury.Severity         87837 non-null  object 
 7   Aircraft.damage         85650 non-null  object 
 8   Aircraft.Category       32278 non-null  object 
 9   Registration.Number     87477 non-null  object 
 10  Make                    88774 non-null  object 
 11  Model                   88745 non-null  object 
 12  Amateur.Built           88739 non-null  object 
 13  Number.of.Engines       82782 non-null  float64
 14  Engine.Type             81747 non-null  obj

In [101]:
#dropped_df[dropped_df['Number.of.Engines'] == 0][['Aircraft.Category', 'Event.Date']]
dropped_df[(dropped_df['Number.of.Engines'] == 0) & (dropped_df['Aircraft.Category'] == 'Airplane')]

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Injury.Severity,Aircraft.damage,Aircraft.Category,Registration.Number,...,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,Location_City,Location_State
19931,20001213X25245,Accident,DCA88WA048,1988-03-27,"OXFORD, United Kingdom",United Kingdom,Fatal(1),Destroyed,Airplane,N91590,...,1.0,0.0,0.0,0.0,UNK,NaN,Foreign,NaN,Oxford,United Kingdom
21901,20001213X27315,Accident,MIA89WA071,1988-11-14,"PANAMA CITY, Panama",Panama,Non-Fatal,NaN,Airplane,N205PA,...,0.0,1.0,4.0,222.0,VMC,NaN,Foreign,06-02-1995,Panama City,Panama
26162,20001212X23667,Accident,DCA90WA049,1990-07-27,"BHARU, Malaysia",Malaysia,Non-Fatal,NaN,Airplane,N108WA,...,0.0,1.0,3.0,3.0,UNK,NaN,Foreign,07-02-1995,Bharu,Malaysia
27319,20001212X16387,Accident,SEA91TA03,1991-01-25,"PINEDALE, WY",United States,Non-Fatal,Substantial,Airplane,N8372Z,...,0.0,0.0,0.0,1.0,UNK,Landing,Probable Cause,31-01-2018,Pinedale,WY
29591,20001212X18577,Incident,CHI92RA060,1991-12-12,"NAKINA, Canada",Canada,Incident,Minor,Airplane,N475EV,...,0.0,0.0,0.0,0.0,VMC,NaN,Foreign,05-08-1996,Nakina,Canada
32014,20001211X16199,Accident,LAX93WA090,1992-12-29,"LORETO, MEXICO",Mexico,Non-Fatal,Substantial,Airplane,N4506X,...,0.0,0.0,0.0,2.0,VMC,NaN,Foreign,05-08-1996,Loreto,MEXICO
61006,20060807X01106,Accident,SEA06CA143,2006-07-09,"ENUMCLAW, WA",United States,Non-Fatal,Substantial,Airplane,N43MM,...,0.0,0.0,0.0,1.0,VMC,Landing,Probable Cause,31-10-2006,Enumclaw,WA
64874,20080827X01334,Accident,CHI08CA202,2008-07-05,"Beloit, WI",United States,Non-Fatal,Substantial,Airplane,N58RW,...,0.0,0.0,0.0,1.0,VMC,NaN,The inadequate visual lookout by the driver of...,25-09-2020,Beloit,WI
76788,20150616X50938,Accident,ERA15LA240,2015-06-14,"Pell City, AL",United States,Non-Fatal,Substantial,Airplane,N1272W,...,0.0,0.0,0.0,1.0,VMC,NaN,The inadequate inspection of the airplane's la...,02-02-2021,Pell City,AL


Now we'll drop entries where Make or Model is null

## IN PROGRESS

In [12]:
dropped_df[dropped_df['Number.of.Engines'].isna()][['Make', 'Model', 'Aircraft.Category']]

,Make,Model,Aircraft.Category
4,Cessna,501,NaN
3600,Piccard,AX-6,NaN
3741,Schweizer,2-33A,NaN
3772,Schweizer,SGS 1-26B,NaN
3870,Pratt-Read,PRG-1,NaN
...,...,...,...
88883,Air Tractor,AT502,NaN
88884,Piper,PA-28-151,NaN
88885,Bellanca,7ECA,NaN
88887,Cessna,210N,NaN


Next, we'll look at the Aircraft Category column. There is still useful information here, but a lot of it is missing. We will try to fill in missing values in this column with info from the Number of Engines column, following these rules:
- 1 engine = 'Small Aircraft'
- 2 engines = 'Medium Aircraft'
- more than 2 engines = 'Large Aircraft'

In [13]:
def num_engines_to_size(record):
    if record['Number.of.Engines'] == 1:
        return 'Small Aircraft'
    elif record['Number.of.Engines'] > 2:
        return 'Large Aircraft'
    else:
        return 'Medium Aircraft'